In [19]:
import torch

print("torch version is : ", torch.__version__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

torch version is :  2.7.1+cu118
cuda


In [20]:
import os

BASE_DIR = os.getcwd()

# Go up one level → BenchmarkDatasets
ROOT_DIR = os.path.dirname(BASE_DIR)
print(ROOT_DIR)
# Dataset path
data_dir = os.path.join(
    ROOT_DIR,
    "data",
    "Flower17",
    "Flower17"
)

print(data_dir)


C:\Users\zabeeh\PycharmProjects\DeepLearning\CompleteProjects\BenchmarkDatasets
C:\Users\zabeeh\PycharmProjects\DeepLearning\CompleteProjects\BenchmarkDatasets\data\Flower17\Flower17


In [32]:
from torchvision import transforms, datasets
from torch.utils.data import Dataset, DataLoader


# loading full datatset
dataset_full = datasets.ImageFolder(root=data_dir, transform=None)

train_size = int(0.8*len(dataset_full))
val_size = len(dataset_full) - train_size

from torch.utils.data import  random_split

train_dataset, val_dataset = random_split(    dataset_full, [train_size, val_size])

# Define data augmentation for training
train_transform = transforms.Compose(
    [
        transforms.Resize((48, 48)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(5),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

# Simpler transform for validation (no augmentation)
validation_transform = transforms.Compose(
    [
        transforms.Resize((48, 48)),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])

train_dataset.dataset.transform = train_transform
val_dataset.dataset.transform = train_transform

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=2)

print(len(dataset_full))
print(dataset_full.classes)

images, labels = next(iter(train_loader))
print(images.shape)  # torch.Size([batch_size, 3, 48, 48])
print(labels.shape)  # torch.Size([batch_size])



1360
['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9']
torch.Size([256, 3, 48, 48])
torch.Size([256])


In [33]:
import torch.nn as nn


class DeepCNN(nn.Module):

    def __init__(self):
        super().__init__()

        # Block 1
        self.block1 = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ELU(),

            nn.Conv2d(32, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ELU(),

            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )

        # Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ELU(),

            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ELU(),

            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )

        # Block 3
        self.block3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),

            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ELU(),

            nn.MaxPool2d(2),
            nn.Dropout2d(0.25)
        )

        self.Flatten = nn.Flatten()

        self.fc1 = nn.Sequential(
            nn.Linear(128 * 6 * 6, 64),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.Dropout(0.25)
        )

        self.fc2 = nn.Sequential(
            nn.Linear(64, 64),
            nn.BatchNorm1d(64),
            nn.ELU(),
            nn.Dropout(0.5)
        )

        self.output = nn.Linear(64, 17)

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.Flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.output(x)
        return x


def init_weights(m):
    if isinstance(m, (nn.Conv2d, nn.Linear)):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        if m.bias is not None:
            nn.init.constant_(m.bias, 0)

    elif isinstance(m, (nn.BatchNorm2d, nn.BatchNorm1d)):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)


# model = DeepCNN().to(device)
model = DeepCNN().to(device=device)
print(model)
model.apply(init_weights)

from torchsummary import summary

summary(model, input_size=(3, 48, 48))
# print(model)

DeepCNN(
  (block1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.25, inplace=False)
  )
  (block2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ELU(alpha=1.0)
    (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ELU(alpha=1.0)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilatio

In [34]:
criterion = nn.CrossEntropyLoss()
optimzier = torch.optim.Adam(model.parameters(), lr=0.001)


In [37]:
epochs = 10
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for batch_index, (images, labels) in enumerate(train_loader):
        images,labels = images.to(device),labels.to(device)

        optimzier.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimzier.step()

        running_loss += loss.item()

        # Calculate accuracy
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Print progress every 10 batches
        if batch_index % 10 == 0:
            print(f"Epoch {epoch+1}, Batch {batch_index}, Loss: {loss.item():.4f}")

    # Print epoch statistics
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = 100.0 * correct / total
    print(f"Epoch {epoch+1}/{epochs} - Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

Epoch 1, Batch 0, Loss: 2.1593
Epoch 1/10 - Loss: 2.1834, Accuracy: 29.32%
Epoch 2, Batch 0, Loss: 2.1828
Epoch 2/10 - Loss: 2.1423, Accuracy: 31.99%
Epoch 3, Batch 0, Loss: 2.0082
Epoch 3/10 - Loss: 1.9682, Accuracy: 34.10%
Epoch 4, Batch 0, Loss: 1.9563
Epoch 4/10 - Loss: 1.9611, Accuracy: 35.75%
Epoch 5, Batch 0, Loss: 1.8422
Epoch 5/10 - Loss: 1.9344, Accuracy: 37.50%
Epoch 6, Batch 0, Loss: 1.8796
Epoch 6/10 - Loss: 1.8616, Accuracy: 38.51%
Epoch 7, Batch 0, Loss: 1.8016
Epoch 7/10 - Loss: 1.7523, Accuracy: 42.83%
Epoch 8, Batch 0, Loss: 1.7788
Epoch 8/10 - Loss: 1.7359, Accuracy: 43.38%
Epoch 9, Batch 0, Loss: 1.7084
Epoch 9/10 - Loss: 1.6883, Accuracy: 45.22%
Epoch 10, Batch 0, Loss: 1.6158
Epoch 10/10 - Loss: 1.6860, Accuracy: 45.31%
